Importing Libs

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import re


Web Scraping for Drivers Death Date

In [ ]:
## Checking DataFrame Structure
df = pd.read_csv('F1/drivers.csv')
df.head()

In [ ]:
## Creating a list to storage the URLs
url = list(df['url'])

In [ ]:
death_dates = []

In [ ]:
months = {
    "January": "01", "February": "02", "March": "03", "April": "04",
    "May": "05", "June": "06", "July": "07", "August": "08",
    "September": "09", "October": "10", "November": "11", "December": "12"
}

In [ ]:
for url in df["url"]:
    try:
        time.sleep(random.uniform(1, 3)) ## Avoidind blocks

        page = requests.get(url, headers=headers)

        soup = BeautifulSoup(page.text, "html.parser")

        ## Fiding the correct table sctructure
        infobox = soup.find("table", {"class": "infobox"})
        if infobox:
            ## Finding the row that contains the Death Date info
            death_date_row = infobox.find("th", string=lambda text: text and "Died" in text)
            if death_date_row:
                raw_death_date = death_date_row.find_next("td").text.strip()
                
                # Extracting the dates with the correct format (DD/MM/YYYY)
                match = re.search(r"(\d{1,2}) (\w+) (\d{4})", raw_death_date)
                if match:
                    day, month, year = match.groups()
                    month_num = months.get(month, "00")
                    death_date = f"{day.zfill(2)}-{month_num}-{year}"
                else:
                    death_date = "Unrecognized Date"
            else:
                death_date = "Alive"
        else:
            death_date = "Not found"

    except requests.exceptions.RequestException:
        death_date = "Request error"

    except Exception:
        death_date = "Unrecognized error"

    death_dates.append(death_date)

In [ ]:
## Adding the information to df
df["driverDeath"] = death_dates
df.head()

Saving the DataFrame as a CSV file

In [ ]:
df.to_csv("F1/drivers_update.csv", index=False)